<a href="https://colab.research.google.com/github/Godwin-88/AlgoTrad/blob/main/Pricing_Derivatives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Required Libraries

In [4]:
import numpy as np


In [ ]:

def european_call_price(S0, K, T, r, sigma, N):
    """
    Computes the price of a European call option using Monte Carlo simulation.

    Args:
        S0: Initial stock price.
        K: Strike price.
        T: Time to maturity.
        r: Risk-free interest rate.
        sigma: Volatility.
        N: Number of simulations.

    Returns:
        The price of the European call option.
    """

    # Generate random numbers from a standard normal distribution
    Z = np.random.standard_normal(N)

    # Calculate the stock price at maturity using the geometric Brownian motion model
    ST = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)

    # Calculate the payoff of the call option
    payoff = np.maximum(ST - K, 0)

    # Calculate the present value of the expected payoff
    call_price = np.exp(-r * T) * np.mean(payoff)

    return call_price

# Parameters
S0 = 100
K = 90
T = 1
r = 0.1
sigma = 0.5
N = 1000

# Calculate the call price
call_price = european_call_price(S0, K, T, r, sigma, N)

print(f"The price of the European call option is: {call_price}")

The price of the European call option is: 27.687010623348797


Binomial Tree Construction

In [2]:
def binomial_tree(S_0, T, u, d, N):
  #The binomial tree is constructed based on 5 parameters including the initial stock price,time horizon, upward movement, downard movement and the number of steps.
    S = np.zeros([N + 1, N + 1])  # Underlying price
    for i in range(0, N + 1):
        S[N, i] = S_0 * (u ** (i)) * (d ** (N - i))
    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            S[j, i] = S_0 * (u ** (i)) * (d ** (j - i))
    return S

In [5]:
Stock = binomial_tree(100, 1, 1.2, 0.8, 2)
Stock

array([[100.,   0.,   0.],
       [ 80., 120.,   0.],
       [ 64.,  96., 144.]])

The output above is an array of values of the stock price at each point in time in a lower triangular matrix.

**Tree Extension with Call Option Payoffs**




In [7]:
def binomial_tree_call(S_0, K, T, u, d, N):
  #The function defined above is variant from the previous in that the strike price K is included.
    C = np.zeros([N + 1, N + 1])  # Call prices
    S = np.zeros([N + 1, N + 1])  # Underlying price
    for i in range(0, N + 1):
        C[N, i] = max(S_0 * (u ** (i)) * (d ** (N - i)) - K, 0)
        S[N, i] = S_0 * (u ** (i)) * (d ** (N - i))
    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            S[j, i] = S_0 * (u ** (i)) * (d ** (j - i))
    return S, C
    #The C output will return the call option payoff at maturity with S being the array of stock price values at each pint in time.

In [8]:
Stock, Call = binomial_tree_call(100, 90, 10, 1.5, 0.5, 5)
print("Underlying Price Evolution:\n", Stock)
print("Call Option Payoff:\n", Call)

Underlying Price Evolution:
 [[100.      0.      0.      0.      0.      0.   ]
 [ 50.    150.      0.      0.      0.      0.   ]
 [ 25.     75.    225.      0.      0.      0.   ]
 [ 12.5    37.5   112.5   337.5     0.      0.   ]
 [  6.25   18.75   56.25  168.75  506.25    0.   ]
 [  3.125   9.375  28.125  84.375 253.125 759.375]]
Call Option Payoff:
 [[  0.      0.      0.      0.      0.      0.   ]
 [  0.      0.      0.      0.      0.      0.   ]
 [  0.      0.      0.      0.      0.      0.   ]
 [  0.      0.      0.      0.      0.      0.   ]
 [  0.      0.      0.      0.      0.      0.   ]
 [  0.      0.      0.      0.    163.125 669.375]]


**Risk-Neutral Probabilities and backward induction of Call Option Value**

In [9]:
def binomial_call_full(S_ini, K, T, r, u, d, N):
#r is introduced as the risk free rate for the computation of risk neutral probabilities.
    dt = T / N  # Define time step
    p = (np.exp(r * dt) - d) / (u - d)  # Risk neutral probabilities (probs)
    C = np.zeros([N + 1, N + 1])  # Call prices
    S = np.zeros([N + 1, N + 1])  # Underlying price
    for i in range(0, N + 1):
        C[N, i] = max(S_ini * (u ** (i)) * (d ** (N - i)) - K, 0)
        S[N, i] = S_ini * (u ** (i)) * (d ** (N - i))
    for j in range(N - 1, -1, -1):
        for i in range(0, j + 1):
            C[j, i] = np.exp(-r * dt) * (p * C[j + 1, i + 1] + (1 - p) * C[j + 1, i])
            S[j, i] = S_ini * (u ** (i)) * (d ** (j - i))
    return C[0, 0], C, S

In [10]:
call_price, C, S = binomial_call_full(100, 90, 10, 0, 1.5, 0.5, 7)
print("Underlying Price Evolution:\n", S)
print("Call Option Payoff:\n", C)
print("Call Option Price at t=0: ", "{:.2f}".format(call_price))

Underlying Price Evolution:
 [[1.00000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.00000000e+01 1.50000000e+02 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.50000000e+01 7.50000000e+01 2.25000000e+02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.25000000e+01 3.75000000e+01 1.12500000e+02 3.37500000e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [6.25000000e+00 1.87500000e+01 5.62500000e+01 1.68750000e+02
  5.06250000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.12500000e+00 9.37500000e+00 2.81250000e+01 8.43750000e+01
  2.53125000e+02 7.59375000e+02 0.00000000e+00 0.00000000e+00]
 [1.56250000e+00 4.68750000e+00 1.40625000e+01 4.21875000e+01
  1.26562500e+02 3.79687500e+02 1.13906250e+03 0.00000000e+00]
 [7.81250000e-01 2.34375000e+00 7.03125000e+00 2.10937500e+01
  6.32812500e+01 1.89843750e+02 5.